In [3]:
from gurobipy import Model, GRB, quicksum
import pandas as pd
import numpy as np

def read_basic_data():
    # Excel file names
    travel_times = pd.read_excel('a2_part1.xlsx','Travel Times')
    lines = pd.read_excel('a2_part1.xlsx', 'Lines')

    lines = lines.drop(['Name','Frequency'], axis = 1)
    lines = lines.fillna(0)

    weights = 1 #Each activity has the same importance for us
    
    return travel_times, lines, weights

travel_times, lines, weights = read_basic_data()

In [33]:
    
    # # Headway at Utrecht from Ams/Sch and to Ams/Sch
    # headway_activities[f'headway - 800/3500 arrival'] = model.addVar(vtype=GRB.CONTINUOUS,
    #                                                     name=f"activity_headway_800-3500_arrival")
    # headway_activities[f'headway - 800/3100 arrival'] = model.addVar(vtype=GRB.CONTINUOUS,
    #                                                     name=f"activity_headway_800-3100_arrival")
    # headway_activities[f'headway - 3000/3500 arrival'] = model.addVar(vtype=GRB.CONTINUOUS,
    #                                                     name=f"activity_headway_3000-3500_arrival")
    # headway_activities[f'headway - 3000/3100 arrival'] = model.addVar(vtype=GRB.CONTINUOUS,
    #                                                     name=f"activity_headway_3000-3100_arrival")
    # 
    # headway_activities[f'headway - 800/3500 departure'] = model.addVar(vtype=GRB.CONTINUOUS,
    #                                                     name=f"activity_headway_800-3500_departure")
    # headway_activities[f'headway - 800/3100 departure'] = model.addVar(vtype=GRB.CONTINUOUS,
    #                                                     name=f"activity_headway_800-3100_departure")
    # headway_activities[f'headway - 3000/3500 departure'] = model.addVar(vtype=GRB.CONTINUOUS,
    #                                                     name=f"activity_headway_3000-3500_departure")
    # headway_activities[f'headway - 3000/3100 departure'] = model.addVar(vtype=GRB.CONTINUOUS,
    #                                                     name=f"activity_headway_3000-3100_departure")

In [20]:
def build_model(travel_times, lines, weights):
    T = 30

    # Initialize the Gurobi model
    model = Model("NS_trains")

    line_names = ['800','3000','3100','3500','3900']
    runnning_activites_f = {}
    runnning_activites_b = {}
    dwelling_activities = {}
    transfer_activities = {}
    sync_activities = {}
    headway_activities = {}
    events_pi_f= {}
    events_pi_b= {}

    for k in range(5):
        station_names = lines.iloc[k].tolist()  # Extract station names from the first row
        number = line_names[k]

        # Create Gurobi variables for each possible trip between stations
        for i in range(len(station_names) - 1):
            station1 = str(station_names[i])
            station2 = str(station_names[i+1])

            if station1 == '0' or station2 == '0':
                break
                
            # Running activities forward (raf)
            runnning_activites_f[f'raf-{station1}-{station2}-{number}'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                                        name=f'raf{station1}-{station2}-{number}')
            # Running activities backwards (rab)
            runnning_activites_b[f'rab-{station2}-{station1}-{number}'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                                        name=f'rab{station2}-{station1}-{number}')
            

            if i != 0 and i != len(station_names):
                # Dwelling activities forward (daf)
                dwelling_activities[f'daf-{station1}-{number}'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                        name=f'daf-{station1}-{number}')
                # Dwelling activities backwards (dab)
                dwelling_activities[f'dab-{station1}-{number}'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                        name=f'dab-{station1}-{number}')
        

            # Events
            
            # Arrival events forward (aef)
            events_pi_f[f'aef-{station1}-{station2}-{number}'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                                            name=f'aef-{station1}-{station2}-{number}')
            # Departure events forward (def)
            events_pi_f[f'def-{station1}-{station2}-{number}'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                                            name=f'def-{station1}-{station2}-{number}')
            # Arrival events forward (aeb)
            events_pi_b[f'aeb-{station2}-{station1}-{number}'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                                            name=f'aeb-{station2}-{station1}-{number}')
            
            # Departure events backwards (deb)
            events_pi_b[f'deb-{station2}-{station1}-{number}'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                                            name=f'deb-{station2}-{station1}-{number}')
            
            
    # Four Transfer activities at Eindhoven
    # Transfer activities backwards (tab)    
    transfer_activities[f'tab-Hrl-Ut-3500'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                        name=f'tab-Hrl-Ut-3500')
    transfer_activities[f'tab-Hrl-Ut-800'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                        name=f'tab-Hrl-Ut-800')
    
    # Transfer activities forward (taf)    
    transfer_activities[f'taf-Ut-Hrl-800'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                        name=f'taf-Ut-Hrl-800')
    transfer_activities[f'taf-Ut-Hrl-3500'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                        name=f'taf-Ut-Hrl-3500')
    
    headway_pairs = [
    ('800', '3500'),
    ('800', '3100'),
    ('3000', '3500'),
    ('3000', '3100')
    ]
    
    #Headway activities arrival or departure
    activity_types = ['a', 'd']
    
    # ha (a/d)
    for pair in headway_pairs:
        for activity_type in activity_types:
            key = f'ha{activity_type}-{pair[0]}-{pair[1]}'
            headway_activities[key] = model.addVar(vtype=GRB.CONTINUOUS, name=key)
    
    
    # Sync activities forwards/backwards (saf/sab)
    sync_pairs = [
    ('Amr_800', 'Asd_3000'),
    ('Asd_800', 'Ut_3000'),
    ('800', '3500'),
    ('3100', '3500'),
    ('3000', '3100'),
    ('800', '3900')
    ]

    # Define the types of headway activities
    direction = ['f', 'b']
    
    # Iterate over each pair and activity type to create variables
    for pair in sync_pairs:
        for d in direction:
            key = f'sa{d}-{pair[0]}-{pair[1]}'
            sync_activities[key] = model.addVar(vtype=GRB.CONTINUOUS, name=key)


    # Objective function: Minimize the total weighted travel time
    model.setObjective(
    quicksum(runnning_activites_f.values()) + 
    quicksum(dwelling_activities.values()) + 
    quicksum(transfer_activities.values()) + 
    quicksum(headway_activities.values()) + 
    quicksum(sync_activities.values()) + 
    quicksum(runnning_activites_b.values()), GRB.MINIMIZE)
    
    # Constrains
    
    # dwelling time constrains 2 < activity < 8
    for key,value in dwelling_activities.items():
        model.addConstr(2 <= dwelling_activities[key],
                name=f'{key}_constrain_2')
        model.addConstr(dwelling_activities[key] <= 8,
                name=f'{key}_constrain_8')
        
    T_pij = 1
    
    for k, activity_var in runnning_activites_f.items():
        # Parse activity keys to get the station names and number
        parts = k.split('-')
        station1, station2, number = parts[1], parts[2], parts[3]
        
        # Find the corresponding events for this activity
        start_event = events_pi_f[f'aef-{station1}-{station2}-{number}']
        end_event = events_pi_f[f'def-{station1}-{station2}-{number}']
        
        # Create the constraint for this activity
        model.addConstr(activity_var == end_event - start_event + T_pij, name=f"duration_{k}")
    
    for k, activity_var in runnning_activites_b.items():
        # Parse activity keys to get the station names and number
        parts = k.split('-')
        station1, station2, number = parts[1], parts[2], parts[3]
        
        # Find the corresponding events for this activity
        start_event = events_pi_b[f'aef-{station1}-{station2}-{number}']
        end_event = events_pi_b[f'def-{station1}-{station2}-{number}']
        
        # Create the constraint for this activity
        model.addConstr(activity_var == end_event - start_event + T_pij, name=f"duration_{k}")

        
        
    
    
    
        

    
    
    
    
    model.update()
    
    return model

model = build_model(travel_times, lines, weights)


KeyError: 'aef-Asd-Amr-800'